In [79]:
import numpy as np
import pandas as pd
import networkx as nx

In [80]:
fp=(eval(input('请输入旗点纵坐标（从0开始）：')), eval(input('请输入旗点横坐标（从0开始）：')))

mat=pd.read_excel(r'C:/Anaconda/JupyterCode/色彩翻翻求解器/3-6 (2,1).xlsx', header=None)
sl=np.shape(mat)[0]

mat

请输入旗点纵坐标（从0开始）：2
请输入旗点横坐标（从0开始）：1


,0,1,2,3,4,5,6,7,8,9,10
0,r,r,y,y,g,b,r,o,r,b,b
1,b,p,y,p,o,o,p,o,p,g,o
2,g,y,y,g,r,b,o,r,r,o,g
3,b,b,b,b,p,b,o,r,o,b,y
4,p,b,b,g,r,b,g,g,g,b,y
5,p,p,p,g,r,y,b,r,p,b,b
6,b,y,b,r,g,p,b,b,p,o,g
7,y,g,b,r,r,r,y,y,r,r,g
8,b,o,g,y,r,y,p,p,p,p,p
9,r,o,o,r,g,y,p,b,r,r,p


In [81]:
G=nx.Graph()

# 添加节点，为其设置颜色属性
for i in range(sl):
    for j in range(sl):
        G.add_node((i,j), color=mat.iloc[i,j])

# 添加边，设置边权
for i in range(sl):
    for j in range(sl):
        if j+1<=sl-1:
            G.add_edge((i,j), (i,j+1), weight=1)
        if i+1<=sl-1:
            G.add_edge((i,j), (i+1,j), weight=1)

# print(G._node)
# print(pd.DataFrame(G.edges(data=True)))

In [82]:
def merge(G, fp):
    G1=G.copy()
    for node in list(G1._node):
        if node!=fp:
            for neighbor in G1[node]:
                if (G1._node[node])['color']==(G1._node[neighbor])['color']:
                    target_neighbor=neighbor
                    other_neighbors=list(set(G1[node])-{target_neighbor})
                        
                    for curr_neighbor in other_neighbors:
                        G1.add_edge(curr_neighbor, target_neighbor, weight=1)
                            
                    G1.remove_node(node)
                    break
                    
    return G1

In [83]:
G=merge(G, fp)

# print(G._node)
# print(pd.DataFrame(G.edges(data=True)))

In [84]:
def calc_TWD(G, fp, dwc=1):
    TWD=0
    for node in G._node:
        depth=nx.dijkstra_path_length(G, source=fp, target=node)
        TWD+=depth*dwc**depth
        
    return TWD

In [85]:
def recursive_min_TWD_calculator(G, fp, dwc, steps, operation):
    G1=G.copy()
    curr_operation=operation
    
    (G1._node[fp])['color']=curr_operation
    G1=merge(G1, fp)
    
    if steps==1:
        return calc_TWD(G1, fp, dwc)
        
    return np.min([recursive_min_TWD_calculator(G1, fp, dwc, steps-1, operation) \
                   for operation in list({'r','o','y','g','b','p'}-{curr_operation})])

In [86]:
def multistep_greedy_algorithm(G, fp, dwc=1, steps=1):
    G1=G.copy()
    pre_TWD=calc_TWD(G1, fp, dwc)
    seq_opt=[]
    
    while pre_TWD>0:
        o_T_df=pd.DataFrame([list({'r','o','y','g','b','p'}-{(G1._node[fp])['color']}), np.empty(5, dtype=int)], \
                            index=['operation', 'TWD']).T
        
        steps_to_use=steps
        is_excessive=True
        while is_excessive==True:
            for i in range(5):
                operation=o_T_df.iloc[i, 0]
                o_T_df.iloc[i, 1]=recursive_min_TWD_calculator(G1, fp, dwc, steps_to_use, operation)
                
            zero_counter=0
            for i in o_T_df['TWD']:
                zero_counter+=1 if i==0 else 0
                
            is_excessive=False if zero_counter<=1 else True
            steps_to_use-=1
            
        operation_opt=o_T_df.sort_values(by='TWD').iloc[0, 0]
        seq_opt.append(operation_opt)
        
        (G1._node[fp])['color']=operation_opt
        G1=merge(G1, fp)
        pre_TWD=calc_TWD(G1, fp, dwc)
        
    return len(seq_opt), seq_opt

In [92]:
steps, seq_opt=multistep_greedy_algorithm(G, fp, dwc=1, steps=6)

print(steps)
print(seq_opt)

14
['b', 'p', 'b', 'g', 'r', 'o', 'y', 'p', 'r', 'b', 'o', 'g', 'p', 'y']
